# Projeto de ***grafos***

Grupo 5:

*   Allane Oliveira
*   Karolayne Melo
*   Pedro Bezerra
*   Samuel Silva
*   Vitor Alencar




## Pacotes e Bibliotecas

In [1]:
!pip install openpyxl pandas numpy

In [46]:
import pandas as pd
import numpy as np
import openpyxl
import math

## Grafos utilizados e funções auxiliares

### Grafos

Para conseguir representar as matrizes de adjacência dos grafos, foi utilizado o recurso DataFrames da Biblioteca Pandas.

In [3]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")

### Funções Auxiliares

#### Função Calcular custo


A função `calcular_custo` serve para somar os pesos das arestas percorridas por um caminho no grafo, funcionando como a medida de qualidade das soluções no algoritmo genético ou memético.

Além disso, ela recebe uma lista de vértices que representa o percurso e percorre essa lista par a par, pegando o custo de cada aresta diretamente na matriz de adjacência do grafo. A cada passo, o custo acumulado é atualizado até que todo o caminho seja percorrido, retornando ao final o custo total.
##### Exemplo:
Em um grafo onde a aresta entre 1 -> 3 custa 2 e 3 -> 2 custa 3, a solução `[1,2,3]` vai gerar um custo final de valor 5.


In [8]:
# Função para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

#### Busca Local Melhoria 1









A função busca_local_primeira_melhoria realiza uma busca local utilizando a estratégia de primeira melhoria. Ela recebe uma solução inicial e gera vizinhos usando a heurística escolhida (como shift, swap ou inversão), avaliando cada vizinho assim que é produzido. Sempre que encontra um vizinho com custo menor que o da solução atual, ela imediatamente adota esse vizinho como nova solução, sem analisar os demais.

O processo se repete enquanto houver melhoria. Quando nenhum vizinho apresenta um custo melhor, a busca é encerrada. Ao final, a função retorna a melhor solução encontrada e seu custo correspondente.

##### Exemplo:
Se um vizinho reduzir o custo de 40 para 28, ele é aceito na hora, e a busca continua a partir dele.

In [7]:
# Executa busca local usando a estratégia de primeira melhoria.
# 'heuristica' é a função que gera vizinhos (shift, swap, inversao).
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False

        # Gera soluções vizinhas usando a heurística escolhida (shift, swap etc.)
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)

            # Se a vizinha for melhor, atualiza e reinicia a busca
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break

    return solucao_atual, custo_atual

#### Heurísticas

As heurísticas apresentadas funcionam como geradores (yield) responsáveis por produzir, uma a uma, novas soluções vizinhas a partir do caminho atual. Cada heurística recebe a solução vigente e cria pequenas variações estruturais que são avaliadas pela função de busca local busca_local_primeira_melhoria. Essas variações permitem que o algoritmo explore alternativas de percurso com potencial para reduzir o custo total.

Todas utilizam uma cópia da solução original, mantêm o primeiro e o último vértice fixos (origem e destino) e realizam alterações apenas na parte interna da solução, acessada por range(1, len(solucao)-1).

##### Heurísticas Implementadas

* **Shift:** Move um vértice interno para outra posição do caminho, produzindo uma nova ordem parcial.

* **Swap:** Troca a posição entre dois vértices internos, criando pequenas permutações da solução.

* **Inversão:** Seleciona uma subsequência interna e inverte sua ordem, gerando modificações maiores, semelhantes ao movimento 2-opt.



In [9]:
# Heurísticas que geram uma vizinhança (usam yield)
# Sempre preservam o primeiro e último vértice do caminho

def shift(solucao: list):
    # Move um vértice para outra posição
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            # Cópia para não alterar a original
            solucao_cp = solucao[:]
            # Remove o vértice da posição atual
            vert_shift = solucao_cp.pop(idx_vert_shift)
            # Insere em outra posição
            solucao_cp.insert(idx_destino_shift, vert_shift)
            # Retorna nova solução
            yield solucao_cp


def swap(solucao: list):
    # Troca dois vértices de posição
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            # Troca simples entre duas posições
            solucao_cp[idx_vert_swap], solucao_cp[idx_destino_swap] = (
                solucao_cp[idx_destino_swap],
                solucao_cp[idx_vert_swap]
            )
            yield solucao_cp


def inversao(solucao: list):
    # Inverte um segmento interno do caminho
    for idx_inicio in range(1, len(solucao) - 1):
        for idx_fim in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            # Segmento a inverter
            sublista = solucao_cp[idx_inicio:idx_fim]
            sublista.reverse()
            # Reconstrói a solução com o segmento invertido
            yield solucao_cp[:idx_inicio] + sublista + solucao_cp[idx_fim:]


## Busca por vizinho mais próximo

A função `*vizinho_mais_proximo*` implementa a heurística clássica do vizinho mais próximo para construir uma solução inicial. A ideia é sempre escolher, a partir do vértice atual, o vértice ainda não visitado que possui o menor custo na matriz do grafo. Dessa forma, ela gera um caminho completo de forma gulosa, visitando todos os vértices uma única vez e retornando ao início no final.

Na implementação, a função começa criando o conjunto de vértices não visitados e inicializando o percurso com o vértice de partida. Em cada iteração, ela busca dentro desse conjunto o vértice de menor custo em relação ao vértice atual, utilizando *idxmin* para localizar o índice correspondente na matriz. O percurso é atualizado acrescentando esse vizinho, o custo total é incrementado, e o vértice atual passa a ser o último escolhido. Quando todos os vértices são visitados, a função adiciona o custo de retorno ao vértice inicial, finalizando o ciclo Hamiltoniano.

O resultado retornado consiste na rota completa construída pela heurística e o custo total acumulado.

#### Exemplo:
Se o vértice 1 tem custo mínimo para o vértice 3, e em seguida o vértice 3 tem custo mínimo para o 2, o algoritmo seguirá essa sequência até visitar todos os nós.

In [11]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total

resultados_vmp = [None] * 12
resultados_vmp_busca_local = [None] * 12

In [12]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out=True):
    # Solução inicial via heurística construtiva
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_caminho, melhor_custo = caminho, custo
    heuristica_melhor_solucao = None

    if out:
        print(f"Aplicando Heurística Construtiva: {heuristica_construtiva.__name__}")
        print(f"Caminho inicial: {caminho}")
        print(f"Custo inicial: {custo}")
        print("\nAplicando Busca Local:")

    # Aplicação das heurísticas de busca local
    for heuristica in heuristicas_busca_local:
        solucao, custo_local = busca_local_primeira_melhoria(
            melhor_caminho, melhor_custo, heuristica, grafo
        )

        if out:
            print(f"\nHeurística: {heuristica.__name__}")
            print("Método: Primeira Melhoria")
            print(f"Caminho obtido: {solucao}")
            print(f"Custo obtido: {custo_local}")

        # Atualiza melhor solução
        if custo_local < melhor_custo:
            melhor_caminho, melhor_custo = solucao, custo_local
            heuristica_melhor_solucao = heuristica.__name__

    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problemas


#### Problema 1

Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em km.

In [59]:
# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp, _ = vizinho_mais_proximo(matriz_problema_km, 1)
resultados_vmp[0] = caminho_vmp

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[0] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo inicial: 2291.3999999999996

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 31, 7, 19, 34, 16, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 17, 25, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45, 1]
Custo obtido: 2222.5999999999995

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 25, 31, 7, 19, 34, 16, 33, 39, 35, 42, 26, 32, 17, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 28, 15, 18, 45, 44, 1]
Custo obtido: 2128.2

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 48, 40, 36, 25, 31, 7, 19, 

#### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.

In [29]:
# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp, _ = vizinho_mais_proximo(matriz_problema_min, 1)
resultados_vmp[1] = caminho_vmp

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[1] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo inicial: 2355.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 33, 20, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 47, 39, 1]
Custo obtido: 2291.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 7, 31, 36, 22, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 5, 47, 38, 1]
Custo obtido: 2230.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 23, 41, 30, 6, 27, 44, 28, 15, 18, 45, 46, 14, 43, 29, 24, 5, 2, 11, 10, 12, 9, 8, 2

#### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [27]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_36, custo_vmp_36 = vizinho_mais_proximo(matriz_problema_km_36, 1)
resultados_vmp[2] = caminho_vmp_36

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[2] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo inicial: 1951.4999999999998

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 36, 25, 31, 7, 34, 16, 33, 20, 17, 19, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30, 1]
Custo obtido: 1781.9999999999998

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 36, 25, 31, 17, 7, 34, 16, 33, 20, 19, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8, 1]
Custo obtido: 1772.8999999999996

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8, 1]
Custo obtido: 1716.8999999999999

Melhor Escolha:

Melhor camin

#### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto em minutos




In [26]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_36, custo_vmp_36 = vizinho_mais_proximo(matriz_problema_min_36, 1)
resultados_vmp[3] = caminho_vmp_36

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[3] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo inicial: 1950.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 25, 31, 7, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 17, 20, 1]
Custo obtido: 1876.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 25, 17, 20, 1]
Custo obtido: 1859.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 30, 6, 27, 28, 15, 18, 14, 29, 24, 13, 35, 33, 16, 34, 26, 32, 19, 7, 31, 36, 22, 25, 17, 20, 1]
Custo obtido: 1728.0

Melhor Escolha:

Melhor caminho final: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 

#### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [25]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_24, custo_vmp_24 = vizinho_mais_proximo(matriz_problema_km_24, 1)
resultados_vmp[4] = caminho_vmp_24

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[4] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo inicial: 1379.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo obtido: 1352.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1337.3000000000004

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1328.6000000000001

Melhor Escolha:

Melhor caminho final: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Melhor Custo:  1328.6000000000001
Heurística responsável: inversao


#### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [31]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_24, custo_vmp_24 = vizinho_mais_proximo(matriz_problema_min_24, 1)
resultados_vmp[5] = caminho_vmp_24

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[5] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Custo inicial: 1283.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 1]
Custo obtido: 1255.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1242.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Custo obtido: 1235.0

Melhor Escolha:

Melhor caminho final: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Melhor Custo:  1235.0
Heurística responsável: inversao


#### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [32]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_12, custo_vmp_12 = vizinho_mais_proximo(matriz_problema_km_12, 1)
resultados_vmp[6] = caminho_vmp_12

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[6] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 10, 12, 11, 2, 3, 4, 5, 7, 6, 1]
Custo inicial: 828.4

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 2, 3, 4, 6, 7, 5, 1]
Custo obtido: 772.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 10, 11, 2, 3, 4, 6, 12, 7, 5, 1]
Custo obtido: 745.7

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Custo obtido: 707.9000000000001

Melhor Escolha:

Melhor caminho final: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Melhor Custo:  707.9000000000001
Heurística responsável: inversao


#### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [33]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_vmp_12, custo_vmp_12 = vizinho_mais_proximo(matriz_problema_min_12, 1)
resultados_vmp[7] = caminho_vmp_12

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12, 1, vizinho_mais_proximo, [swap, shift, inversao])
resultados_vmp_busca_local[7] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 6, 7, 1]
Custo inicial: 687.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 8, 1]
Custo obtido: 649.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Custo obtido: 609.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Custo obtido: 609.0

Melhor Escolha:

Melhor caminho final: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Melhor Custo:  609.0
Heurística responsável: shift


#### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km

In [37]:
# Seleção das cidades específicas
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Criação da submatriz
matriz_problema_km_7 = matriz_problema_km[cidades_7].iloc[indices_7].copy()
matriz_problema_km_7.index = range(len(cidades_7))
matriz_problema_km_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial (Vizinho Mais Próximo)
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_km_7, 1)
resultados_vmp[8] = caminho_inicial

# Aplicando busca local com as heurísticas
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_7,1,vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[8] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 5, 7, 6, 2, 1]
Custo inicial: 518.5

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Melhor Escolha:

Melhor caminho final: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor Custo:  438.3
Heurística responsável: swap


#### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [35]:
# Seleção das cidades específicas
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Criação da submatriz
matriz_problema_min_7 = matriz_problema_min[cidades_7].iloc[indices_7].copy()
matriz_problema_min_7.index = range(len(cidades_7))
matriz_problema_min_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial (Vizinho Mais Próximo)
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_min_7, 1)
resultados_vmp[9] = caminho_inicial

# Aplicando busca local com as heurísticas
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_7,1,vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[9] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 7, 5, 6, 2, 1]
Custo inicial: 413.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Melhor Escolha:

Melhor caminho final: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor Custo:  364.0
Heurística responsável: swap


#### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [38]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1,7))].iloc[0:6].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_km_6, 1)
resultados_vmp[10] = caminho_inicial

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[10] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 2, 5, 6, 1]
Custo inicial: 463.6

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 348.3

Melhor Escolha:

Melhor caminho final: [1, 5, 2, 3, 4, 6, 1]
Melhor Custo:  348.3
Heurística responsável: swap


#### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos


In [39]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1,7))].iloc[0:6].copy()

# Caminho inicial gerado pelo Vizinho Mais Próximo
caminho_inicial, custo_inicial = vizinho_mais_proximo(matriz_problema_min_6, 1)
resultados_vmp[11] = caminho_inicial

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6, 1, vizinho_mais_proximo,[swap, shift, inversao])
resultados_vmp_busca_local[11] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo: ", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: vizinho_mais_proximo
Caminho inicial: [1, 3, 4, 2, 5, 6, 1]
Custo inicial: 400.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 5, 2, 3, 4, 6, 1]
Custo obtido: 305.0

Melhor Escolha:

Melhor caminho final: [1, 5, 2, 3, 4, 6, 1]
Melhor Custo:  305.0
Heurística responsável: swap


## Inserção mais barata

Vamos aplicar a inserção e aplicar nos problemas.

A função `insercao_mais_barata` constrói uma rota inicial usando a ideia de sempre inserir o próximo vértice no ponto onde ele aumenta o custo o mínimo possível. Primeiro ela monta um ciclo simples com o vértice inicial e o vértice mais próximo dele. Depois, enquanto ainda houver vértices de fora, escolhe sempre o vértice mais próximo de qualquer vértice já presente na rota.

Para cada vértice escolhido, o algoritmo testa todas as posições possíveis dentro da rota e calcula quanto o custo aumentaria caso ele fosse inserido ali. A posição que causa o menor aumento é usada. Isso continua até que todos os vértices tenham sido incluídos na rota.

No final, a função retorna a rota construída e o custo total calculado.

In [40]:
# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = int(grafo[vertice_inicial].idxmin())

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = int(grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin())

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)

resultados_imb = [None] * 12
resultados_imb_busca_local = [None] * 12


In [41]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out=True):
    # Solução inicial via heurística construtiva
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_caminho, melhor_custo = caminho, custo
    heuristica_melhor_solucao = None

    if out:
        print(f"Aplicando Heurística Construtiva: {heuristica_construtiva.__name__}")
        print(f"Caminho inicial: {caminho}")
        print(f"Custo inicial: {custo}")
        print("\nAplicando Busca Local:")

    # Aplicação das heurísticas de busca local
    for heuristica in heuristicas_busca_local:
        solucao, custo_local = busca_local_primeira_melhoria(
            melhor_caminho, melhor_custo, heuristica, grafo
        )

        if out:
            print(f"\nHeurística: {heuristica.__name__}")
            print("Método: Primeira Melhoria")
            print(f"Caminho obtido: {solucao}")
            print(f"Custo obtido: {custo_local}")

        # Atualiza melhor solução
        if custo_local < melhor_custo:
            melhor_caminho, melhor_custo = solucao, custo_local
            heuristica_melhor_solucao = heuristica.__name__

    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problemas

#### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km

In [47]:
# Caminho inicial gerado pela heurística de Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km, 1)
resultados_imb[0] = caminho_imb

# Busca local usando swap, shift e inversão após Inserção Mais Barata
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km,  1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[0] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo:", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo inicial: 2148.1

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 11, 10, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo obtido: 2115.5999999999995

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 10, 12, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 11, 2, 23, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo obtido: 2078.2999999999997

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 2

#### Problema 2

Percurso por 48 cidades, partindo de ANGICOS, com função custo definida pelo tempo

In [48]:
# Caminho inicial gerado pela heurística de Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min, 1)
resultados_imb[1] = caminho_imb

# Busca local usando swap, shift e inversão após Inserção Mais Barata
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min,  1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[1] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho final:", melhor_caminho)
print("Melhor Custo:", melhor_custo)
print("Heurística responsável:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 33, 39, 35, 42, 32, 26, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 3, 4, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo inicial: 2197.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 33, 35, 42, 26, 32, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 4, 3, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo obtido: 2167.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 16, 34, 33, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 3, 4, 2, 11, 5, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo obtido: 2119.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 38, 20, 17, 25, 40, 48, 22, 36, 31, 7, 19, 33, 34, 16, 32, 26, 42, 35, 39, 4

#### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [49]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_36, 1)
resultados_imb[2] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[2] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 21, 9, 12, 10, 11, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo inicial: 1835.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 11, 10, 12, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo obtido: 1814.8999999999999

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 21, 9, 10, 12, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo obtido: 1786.6000000000001

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 4, 3, 2, 11, 5, 23, 1]
Custo obtido: 1743.2

Melhor Escolha:

Melhor caminho: [1, 10, 12, 9, 8, 21

#### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto em minutos

In [50]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_36, 1)
resultados_imb[3] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[3] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo inicial: 1931.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1931.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 11, 5, 14, 29, 24, 13, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1828.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 10, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo obtido: 1765.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 12, 10, 20, 17, 25, 22, 36, 31, 7,

#### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [51]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_24, 1)
resultados_imb[4] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[4] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 10, 11, 21, 22, 17, 7, 16, 19, 20, 13, 24, 14, 5, 2, 3, 4, 6, 15, 18, 23, 1]
Custo inicial: 1459.3000000000002

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1405.1000000000001

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1390.5000000000002

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Custo obtido: 1390.5000000000002

Melhor Escolha:

Melhor caminho: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo: 1390.5000000000002
Heurística: shift


#### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [52]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_24, 1)
resultados_imb[5] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[5] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 12, 24, 13, 20, 19, 16, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Custo inicial: 1355.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Custo obtido: 1351.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Custo obtido: 1260.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Custo obtido: 1260.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Melhor custo: 1260.0
Heurística: shift


#### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [53]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_12, 1)
resultados_imb[6] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[6] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo inicial: 711.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 711.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 711.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Custo obtido: 699.2

Melhor Escolha:

Melhor caminho: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Melhor custo: 699.2
Heurística: inversao


#### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [54]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_12, 1)
resultados_imb[7] = caminho_imb

# Busca local (swap, shift e inversão)
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12,1,insercao_mais_barata,[swap, shift, inversao])
resultados_imb_busca_local[7] = melhor_caminho

print("\nMelhor Escolha:")
print("\nMelhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)

Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo inicial: 609.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo obtido: 609.0

Melhor Escolha:

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo: 609.0
Heurística: None


#### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km


In [55]:
# cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# indices_7 = [c - 1 for c in cidades_7]

# sub_matriz = matriz_problema_km[cidades_7].iloc[indices_7].copy()
# mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

# matriz_normalizada = sub_matriz.reset_index(drop=True)
# matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

# melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
#     matriz_normalizada,
#     1,
#     insercao_mais_barata,
#     [swap, shift, inversao]
# )

# melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

# print(f"\nMelhor caminho: {melhor_caminho_real}")
# print(f"Melhor custo: {melhor_custo}")
# print(f"Heurística: {heuristica_melhor_solucao}")

# resultados_imb[8] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
# resultados_imb_busca_local[8] = melhor_caminho_real

# Seleção das cidades [1, 7, 8, 9, 10, 11, 12]

cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Filtra a matriz de acordo com as cidades selecionadas
matriz_problema_km_7 = matriz_problema_km[cidades_7].iloc[indices_7].copy()

# Ajusta índices e colunas para valores sequenciais
matriz_problema_km_7.index = range(len(cidades_7))
matriz_problema_km_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_7, 1)
resultados_imb[8] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_7, 1, insercao_mais_barata, [swap, shift, inversao])

# Armazena resultados da busca local
resultados_imb_busca_local[8] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)



Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 2, 7, 5, 6, 1]
Custo inicial: 438.3

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 438.3

Melhor Escolha:
Melhor caminho: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo: 438.3
Heurística: None


#### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [56]:
# cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# indices_7 = [c - 1 for c in cidades_7]

# sub_matriz = matriz_problema_min[cidades_7].iloc[indices_7].copy()

# mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

# matriz_normalizada = sub_matriz.reset_index(drop=True)
# matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

# melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
#     matriz_normalizada,
#     1,
#     insercao_mais_barata,
#     [swap, shift, inversao]
# )

# melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

# print(f"\nMelhor caminho (Original): {melhor_caminho_real}")
# print(f"Melhor custo: {melhor_custo}")
# print(f"Heurística vencedora: {heuristica_melhor_solucao}")

# resultados_imb[9] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
# resultados_imb_busca_local[9] = melhor_caminho_real

# Seleção das cidades [1, 7, 8, 9, 10, 11, 12]
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

# Filtra a matriz de acordo com as cidades selecionadas
matriz_problema_min_7 = matriz_problema_min[cidades_7].iloc[indices_7].copy()

# Ajusta índices e colunas para valores sequenciais
matriz_problema_min_7.index = range(len(cidades_7))
matriz_problema_min_7.columns = range(1, len(cidades_7) + 1)

# Caminho inicial pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_7, 1)
resultados_imb[9] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_problema_min_7,
    1,
    insercao_mais_barata,
    [swap, shift, inversao]
)

# Armazena resultados da busca local
resultados_imb_busca_local[9] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 2, 7, 5, 6, 1]
Custo inicial: 364.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 2, 7, 5, 6, 1]
Custo obtido: 364.0

Melhor Escolha:
Melhor caminho: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo: 364.0
Heurística: None


#### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [57]:
# Filtra as colunas e linhas para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1, 7))].iloc[0:6].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_km_6, 1)
resultados_imb[10] = caminho_imb

# Busca local com swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, insercao_mais_barata, [swap, shift, inversao])
resultados_imb_busca_local[10] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 6, 2, 5, 1]
Custo inicial: 348.5

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 6, 4, 3, 2, 5, 1]
Custo obtido: 348.3

Melhor Escolha:
Melhor caminho: [1, 6, 4, 3, 2, 5, 1]
Melhor custo: 348.3
Heurística: swap


#### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [58]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1, 7))].iloc[0:6].copy()

# Caminho inicial gerado pela Inserção Mais Barata
caminho_imb, _ = insercao_mais_barata(matriz_problema_min_6, 1)
resultados_imb[11] = caminho_imb

# Busca local usando swap, shift e inversão
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6,  1,  insercao_mais_barata,  [swap, shift, inversao])
resultados_imb_busca_local[11] = melhor_caminho

print("\nMelhor Escolha:")
print("Melhor caminho:", melhor_caminho)
print("Melhor custo:", melhor_custo)
print("Heurística:", heuristica_melhor_solucao)


Aplicando Heurística Construtiva: insercao_mais_barata
Caminho inicial: [1, 3, 4, 6, 2, 5, 1]
Custo inicial: 305.0

Aplicando Busca Local:

Heurística: swap
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Heurística: shift
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Heurística: inversao
Método: Primeira Melhoria
Caminho obtido: [1, 3, 4, 6, 2, 5, 1]
Custo obtido: 305.0

Melhor Escolha:
Melhor caminho: [1, 3, 4, 6, 2, 5, 1]
Melhor custo: 305.0
Heurística: None


## A seguir o link para o próximo notebook de [Algoritmo Genético e Memetico](https://)